## imports

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

## Preprocessing and augmenting

In [4]:
# Define image size and dataset path
img_height, img_width = 224, 224  # ResNet50 requires 224x224 input size
path_to_images = 'Intel-Image-Subset'

# Define the categories to load
categories = ['building', 'forest', 'mountain', 'sea']

# Load and preprocess training images with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(path_to_images, 'train'),
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Load and preprocess test and validation images (no augmentation, grayscale conversion)
test_datagen = ImageDataGenerator(rescale=1./255)  # Only normalize the pixel values
val_datagen = ImageDataGenerator(rescale=1./255)   # Only normalize the pixel values

test_generator = test_datagen.flow_from_directory(
    os.path.join(path_to_images, 'test'),
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Don't shuffle test set
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(path_to_images, 'val'),
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Don't shuffle validation set
)

# Define the ResNet50 model for feature extraction (excluding the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Use the ResNet50 model to extract features from the images
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Function to extract features using the ResNet50 model
def extract_features(generator):
    features = []
    labels = []
    for i in range(generator.samples // generator.batch_size):
        x_batch, y_batch = next(generator)  # Corrected to use 'next()' on generator
        feature_batch = feature_extractor.predict(x_batch)  # Extract features from the batch
        features.append(feature_batch)
        labels.append(y_batch)
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

# Extract features from training, test, and validation sets
train_features, train_labels = extract_features(train_generator)
test_features, test_labels = extract_features(test_generator)
val_features, val_labels = extract_features(val_generator)

# Flatten the features for use in classifiers
train_features_flattened = train_features.reshape(train_features.shape[0], -1)
test_features_flattened = test_features.reshape(test_features.shape[0], -1)
val_features_flattened = val_features.reshape(val_features.shape[0], -1)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_features_flattened)
X_test_scaled = scaler.transform(test_features_flattened)
X_val_scaled = scaler.transform(val_features_flattened)

pca = PCA(n_components=0.80)  # Keep 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)

mi = mutual_info_classif(X_train_pca, train_labels.argmax(axis=1))  # Use argmax to get class indices
mi_df = pd.DataFrame(mi, columns=['Mutual Information'])

k = 30  # Adjust k based on how many top features you want to select
top_k_features = np.argsort(mi)[-k:]  # Indices of the k highest MI components

X_test_pca = pca.transform(X_test_scaled)
X_val_pca = pca.transform(X_val_scaled)

X_train_selected = X_train_pca[:, top_k_features]
X_test_selected = X_test_pca[:, top_k_features]
X_val_selected = X_val_pca[:, top_k_features]

Found 2800 images belonging to 4 classes.
Found 600 images belonging to 4 classes.
Found 600 images belonging to 4 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━

## Params

In [5]:
learning_rate = 0.003
batch_size = 8
epochs = 20

In [ ]:
# Function to create a CNN model with the given hyperparameters
def create_cnn_model(input_shape, learning_rate=0.001):
    model = models.Sequential()

    # First Conv2D layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Second Conv2D layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Optional: Additional Conv2D layer if needed
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output to feed into dense layers
    model.add(layers.Flatten())

    # Fully connected layers (dense layers)
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))  # Output layer with 4 categories

    # Compile the model with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

## Trene en modell
(ikke dene thomas)

In [ ]:
# Assuming `learning_rate`, `batch_size`, and `epochs` are defined earlier
cnn_model = create_cnn_model(input_shape=(img_height, img_width, 3), learning_rate=learning_rate)

# Train the model
history = cnn_model.fit(
    train_generator,  # Use the image generators directly
    epochs=epochs,    # Number of epochs
    batch_size=batch_size,
    validation_data=val_generator  # Validation data for evaluation
)

# Plot the training and validation accuracy and loss curves
plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

# Show both plots
plt.tight_layout()
plt.show()

## Testing hyper parameters.
(Also with one less layer) - men skrev ikke om dette(: tenkte det sparte litt tid. Kan evt bruke create_cnn_model() istedenfor.

In [9]:
# Function to create a CNN model with the given hyperparameters
def create_cnn_model2(input_shape, learning_rate=0.001):
    model = models.Sequential()

    # First Conv2D layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Second Conv2D layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten the output to feed into dense layers
    model.add(layers.Flatten())

    # Fully connected layers (dense layers)
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(4, activation='softmax'))  # Output layer with 4 categories

    # Compile the model with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

### Testing batch size

In [ ]:
batch_sizes = [8, 16, 32, 64]
for bs in batch_sizes:
    model = create_cnn_model2(input_shape=(img_height, img_width, 3), learning_rate=learning_rate)
    history = model.fit(train_generator, epochs=epochs, batch_size=bs, validation_data=val_generator)
    
    plt.figure(figsize=(12, 6))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f"Training and Validation Accuracy, batch_size={bs}")

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # Show both plots
    plt.tight_layout()
    plt.show()

C:\Users\Anine Løkken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.3970 - loss: 4.2901 - val_accuracy: 0.5667 - val_loss: 0.9607
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.5944 - loss: 0.9959 - val_accuracy: 0.6850 - val_loss: 0.7667
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 116s 1s/step - accuracy: 0.6665 - loss: 0.8142 - val_accuracy: 0.6683 - val_loss: 0.8096
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.6689 - loss: 0.8110 - val_accuracy: 0.7283 - val_loss: 0.6827
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.6896 - loss: 0.7429 - val_accuracy: 0.6983 - val_loss: 0.7124
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.6611 - loss: 0.7550 - val_accuracy: 0.7317 - val_loss: 0.6988
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.7085 - loss: 0.6911 - val_accuracy: 0.7400 - val_loss: 0.6524
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.7102 - loss: 0.6846 - val_accuracy: 0.7150 - v

### Testing learing_rate

In [ ]:
learning_rates = [0.001, 0.005, 0.01]

for lr in learning_rates:
    model = create_cnn_model2(input_shape=(img_height, img_width, 3), learning_rate=lr)
    history = model.fit(train_generator, epochs=20, batch_size=16, validation_data=val_generator)
    
    plt.figure(figsize=(12, 6))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f"Training and Validation Accuracy, learning_rate={lr}")

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # Show both plots
    plt.tight_layout()
    plt.show()

### Final model

Når vi har funnet beste batch size og learning rate, kan vi plotte hvordan modellen fungerer med de riktige verdiene her:

In [ ]:
learning_rate_final = 0.005
batch_size_final = 16
epochs_final = 20

In [ ]:
model = create_cnn_model2(input_shape=(img_height, img_width, 3), learning_rate=learning_rate_final)
history = model.fit(train_generator, epochs=epochs, batch_size=batch_size, validation_data=val_generator)

plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title(f"Training and Validation Accuracy,\nlearning_rate={learning_rate_final}, batch_size={batch_size_final}, epochs={epochs_final}")

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title(f"'Training and Validation Loss,\nlearning_rate={learning_rate_final}, batch_size={batch_size_final}, epochs={epochs_final}")

# Show both plots
plt.tight_layout()
plt.show()